# Домашнее задание № 4. Языковые модели

## Задание 1 (8 баллов).

В семинаре для генерации мы использовали предположение маркова и считали, что слово зависит только от 1 предыдущего слова. Но ничто нам не мешает попробовать увеличить размер окна и учитывать два или даже три прошлых слова. Для них мы еще сможем собрать достаточно статистик и, логично предположить, что качество сгенерированного текста должно вырасти.

Попробуйте сделать языковую модель, которая будет учитывать два предыдущих слова при генерации текста.
Сгенерируйте несколько текстов (3-5) и расчитайте перплексию получившейся модели.
Можно использовать данные из семинара или любые другие (можно брать только часть текста, если считается слишком долго). Перплексию рассчитывайте на 10-50 отложенных предложениях (они не должны использоваться при сборе статистик).


Подсказки:  
    - нужно будет добавить еще один тэг \<start>  
    - можете использовать тот же подход с матрицей вероятностей, но по строкам хронить биграмы, а по колонкам униграммы
    - тексты должны быть очень похожи на нормальные (если у вас получается рандомная каша, вы что-то делаете не так)
    - у вас будут словари с индексами биграммов и униграммов, не перепутайте их при переводе индекса в слово - словарь биграммов будет больше словаря униграммов и все индексы из униграммного словаря будут формально подходить для словаря биграммов (не будет ошибки при id2bigram[unigram_id]), но маппинг при этом будет совершенно неправильным

In [ ]:
!pip install razdel

In [ ]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
import random
from collections import Counter

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
news = open('/content/drive/MyDrive/Colab Notebooks/lenta.txt').read()

In [ ]:
news_sent = [sent.text for sent in list(sentenize(news))]
random.shuffle(news_sent)

In [ ]:
news_sent_main = news_sent[50:]
news_sent_test = news_sent[:50]

In [ ]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text

In [ ]:
norm_news = normalize(' '.join(news_sent_main))

In [ ]:
vocab_news = Counter(norm_news)
probas_news = Counter({word:c/len(norm_news) for word, c in vocab_news.items()})

In [ ]:
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0, len(tokens) - n + 1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [ ]:
sentences_news = [['<start1>', '<start2>'] + normalize(text) + ['<end>'] for text in news_sent_main]

In [ ]:
unigrams_news = Counter()
bigrams_news = Counter()
trigrams_news = Counter()

for sentence in sentences_news:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence))
    trigrams_news.update(ngrammer(sentence, 3))

In [ ]:
from scipy.sparse import lil_matrix, csr_matrix, csc_matrix
import numpy as np

In [ ]:
matrix_news = lil_matrix((len(bigrams_news),
                        len(unigrams_news)))

id2word_news = list(unigrams_news)
word2id_news = {word:i for i, word in enumerate(id2word_news)}

id2bigr_news =list(bigrams_news)
bigr2id_news = {bigr:i for i, bigr in enumerate(id2bigr_news)}


for ngram in trigrams_news:
    bigr = ' '.join(ngram.split()[:2])
    word = ngram.split()[2]
    matrix_news[bigr2id_news[bigr], word2id_news[word]] =  (trigrams_news[ngram]/
                                                                     bigrams_news[bigr])

matrix_news = csc_matrix(matrix_news)

In [ ]:
def apply_temperature(probas, temperature):
    # логарифмирование и деление на температуру
    log_probas = np.log(np.maximum(probas, 1e-10))
    adjusted_log_probas = log_probas / temperature
    # чтобы получить честные вероятности, нужно применить софтмакс
    exp_probas = np.exp(adjusted_log_probas)
    adjusted_probabilities = exp_probas / np.sum(exp_probas)
    return adjusted_probabilities

In [ ]:
#версия 1 (без Beam)
def generate(matrix, id2word, bigr2id, n=100, start='<start1> <start2>', temperature=1.2):
    text = []
    current_idx = bigr2id[start]
    current_bigr_split = start.split()

    for i in range(n):
        probas = matrix[current_idx].toarray()[0]

        if probas.sum() == 0:
            chosen = np.random.randint(0, matrix.shape[1])
        else:
            chosen = np.random.choice(
                matrix.shape[1],
                p=apply_temperature(probas, temperature=temperature)
            )

        chosen_word = id2word[chosen]
        text.append(chosen_word)

        if chosen_word == '<end>':
            current_idx = bigr2id[start]
            current_bigr_split = start.split()
        else:
            current_bigr_split = [current_bigr_split[1], chosen_word]
            new_bigr = ' '.join(current_bigr_split)

            if new_bigr in bigr2id:
                current_idx = bigr2id[new_bigr]
            else:
                second_token = current_bigr_split[0]
                candidates = [bigr for bigr in bigr2id.keys() if bigr.split()[1] == second_token]
                if candidates:
                    current_idx = bigr2id[random.choice(candidates)]
                else:
                    try:
                        current_idx = bigr2id[start]
                    except:
                        raise KeyError('Генерация с таким стартом невозможна')

    return ' '.join(text)


In [ ]:
for _ in range(3):
    print(generate(matrix_news, id2word_news, bigr2id_news).replace(' <end>', '.'))

запертые журналисты подняли тревогу и озабоченность заявил исполняющий обязанности министра обороны сообщает reuters в последний путь погибших при взрыве произошедшем в связи с осложнением обстановки в чечне будет амнистирована еще одна небольшая снежная лавина. коллина сообщил представителям уефа о том что китайское правительство разрабатывает новые правила разработаны с целью обнаружения взрывоопасных предметов оставленных боевиками. именно кандидатуру артемьева яблоко собиралось поддерживать на несостоявшихся губернаторских выборах. несмотря на множество встреч и от всей конструкции и пролетел 20 метров. на место происшествия прибыли сорудники скорой помощи им склифосовского 1-ая градская 36-ая и 64-ая горбольница работают только на сумму 325
наконец в мосгоризбирком подал заявление об отставке. как сообщили интерфаксу в аппарате полномочного представителя президента россии самарского губернатора председателя политсовета петербургского движения яблока. леонид кучма решивший обзавест

запертые журналисты подняли тревогу и озабоченность заявил исполняющий обязанности министра обороны сообщает reuters в последний путь погибших при взрыве произошедшем в связи с осложнением обстановки в чечне будет амнистирована еще одна небольшая снежная лавина. коллина сообщил представителям уефа о том что китайское правительство разрабатывает новые правила разработаны с целью обнаружения взрывоопасных предметов оставленных боевиками. именно кандидатуру артемьева яблоко собиралось поддерживать на несостоявшихся губернаторских выборах. несмотря на множество встреч и от всей конструкции и пролетел 20 метров. на место происшествия прибыли сорудники скорой помощи им склифосовского 1-ая градская 36-ая и 64-ая горбольница работают только на сумму 325

наконец в мосгоризбирком подал заявление об отставке. как сообщили интерфаксу в аппарате полномочного представителя президента россии самарского губернатора председателя политсовета петербургского движения яблока. леонид кучма решивший обзавестись собственным сервером в интернете. по мнению аушева назначение кадырова результат действий военного лобби того самого элиакима рубенштейна который в случае положительного решения послужило то что зараза переселяется в компьютер пользователя при посещении сайтов идентифицируют программное обеспечение предназначенное для лечения диабета антибиотики и вакцины. владимир татаренков воглавлял преступную группировку в городе бийске алтайского края почти не отличаются от шансов неожиданно скончаться на земле катастрофа случилась в 21 10 по местному

полностью выгорело 17 тысяч человек. рубин заявил что сотрудники министерства определили что граната учебная. кроме того лидер спс считает бесспорно незаконной. во вторник кабельная компании ntl сообщила что в результате которой участники получили около 50 метров во время планового обхода территории заповедника. питерцы победили со счетом 75 55 31 33 третий решающий матч серии до трех с лишним тысяч рублей. все участковые комиссии в которую опускают бюллетени. виктор черепков он набрал при голосовании по кандидатуре надолжность спикера нижней палаты парламента боснии и герцеговине. церетели сообщил что всего существует около 4,1 миллиона таких счетов


In [ ]:
def perplexity(logp, N):
    return np.exp((-1/N) * logp)

In [ ]:
def compute_joint_proba(text, word_probas):
    prob = 0
    tokens = normalize(text)
    for word in tokens:
        if word in word_probas:
            prob += (np.log(word_probas[word]))
        else:
            prob += np.log(2e-4)

    return prob, len(tokens)

In [ ]:
def compute_join_proba_markov_assumption(text, word_counts, bigram_counts):
    prob = 0
    tokens = normalize(text)
    for ngram in ngrammer(['<start>'] + tokens + ['<end>']):
        word1, word2 = ngram.split()
        if word1 in word_counts and ngram in bigram_counts:
            prob += np.log(bigram_counts[ngram]/word_counts[word1])
        else:
            prob += np.log(2e-5)

    return prob, len(tokens)

In [ ]:
def compute_join_proba_trigram(text, word_counts, bigram_counts, trigram_counts):
    prob = 0
    tokens = normalize(text)
    for ngram in ngrammer(['<start1>', '<start2>'] + tokens + ['<end>'], 3):
        bigr = ' '.join(ngram.split()[:2])
        word = ngram.split()[2]
        if bigr in bigram_counts and ngram in trigram_counts:
            prob += np.log(trigram_counts[ngram]/bigram_counts[bigr])
        else:
            prob += np.log(1e-3)

    return prob, len(tokens)

In [ ]:
ps_1 = []

for sent in news_sent_test:

  prob_1, N_1 = compute_joint_proba(sent, probas_news)
  if not N_1:
        continue

  ps_1.append(perplexity(prob_1, N_1))

In [ ]:
ps_2 = []

for sent in news_sent_test:
    prob_2, N_2 = compute_join_proba_markov_assumption(sent, unigrams_news, bigrams_news)

    if not N_2:
        continue

    ps_2.append(perplexity(prob_2, N_2))

In [ ]:
ps_3 = []

for sent in news_sent_test:

      prob_3, N_3 = compute_join_proba_trigram(sent, unigrams_news, bigrams_news, trigrams_news)
      if not N_3:
        continue

      ps_3.append(perplexity(prob_3, N_3))

In [ ]:
print(np.mean(ps_1))
print(np.mean(ps_2))
print(np.mean(ps_3))

9276.10350000907
3050.868403764363
630.3099085184163


## Задание № 2* (2 балла).

Измените функцию generate_with_beam_search так, чтобы она работала с моделью, которая учитывает два предыдущих слова.
Сравните получаемый результат с первым заданием.
Также попробуйте начинать генерацию не с нуля (подавая \<start> \<start>), а с какого-то промпта. Но помните, что учитываться будут только два последних слова, так что не делайте длинные промпты.

In [ ]:
class Beam:
    def __init__(self, sequence: list, score: float):
        self.sequence: list = sequence
        self.score: float = score

In [ ]:
#версия 2 (c Beam + температура)
def generate_with_beam_search(matrix, id2word, bigr2id, n=100, max_beams=5, start='<start1> <start2>', temperature=1.3):
    # изначально у нас один луч с заданным началом (start по дефолту)
    initial_node = Beam(sequence=[start], score=np.log1p(0))
    beams = [initial_node]

    for i in range(n):
        # делаем n шагов генерации
        new_beams = []
        # на каждом шаге продолжаем каждый из имеющихся лучей
        for beam in beams:
            # лучи которые уже закончены не продолжаем (но и не удаляем)
            if beam.sequence[-1] == '<end>':
                new_beams.append(beam)
                continue


            tokens = []
            for el in beam.sequence:
                tokens.extend(el.split())

            last_bigram = tokens[-2] + ' ' + tokens[-1]

            if last_bigram in bigr2id:
                last_id = bigr2id[last_bigram]
            else:
                second_token = tokens[-1]
                candidates = [bigr for bigr in bigr2id.keys() if bigr.split()[1] == second_token]
                if candidates:
                    last_id = bigr2id[random.choice(candidates)]
                else:
                    try:
                        last_id = bigr2id[start]
                    except:
                        raise KeyError('Генерация с таким стартом невозможна')



            # посмотрим вероятности продолжений для предыдущего слова
            probas = matrix[last_id].toarray()[0]


            if probas.sum() == 0:
                top_idxs = [np.random.randint(0, matrix.shape[1])]
            else:
                #доп обработка! иначе ошибка на высоких температурах
                probas_temp = apply_temperature(probas, temperature=temperature)
                probas_temp /= probas_temp.sum()
                top_idxs = np.random.choice(matrix.shape[1],
                                        size=min(max_beams, probas.astype(bool).sum()),
                                        p=probas_temp, replace=False)

            for top_id in top_idxs:
                # иногда вероятности будут нулевые, такое не добавляем
                if not probas[top_id]:
                    #поменяла на continue с надеждой, что предложения не будут обрываться слишком рано
                    continue

                # создадим новый луч на основе текущего и варианта продолжения
                new_sequence = beam.sequence + [id2word[top_id]]



                if len(new_sequence) > 35:   #они слишком длинные и всегда плохие
                    continue

                #попробуем как-то поощрять длинные предложения
                length_bonus = 0.2 * np.log1p(len(new_sequence))


                new_score = beam.score + np.log(max(probas[top_id], 1e-10)) + length_bonus
                new_beam = Beam(sequence=new_sequence, score=new_score)
                new_beams.append(new_beam)

        # отсортируем лучи по скору и возьмем только топ max_beams
        beams = sorted(new_beams, key=lambda x: x.score, reverse=True)[:max_beams]

    # в конце возвращаем самый вероятный луч
    best_sequence = max(beams, key=lambda x: x.score).sequence



    return ' '.join(best_sequence)



In [ ]:
for _ in range(3):
    print(generate_with_beam_search(matrix_news, id2word_news, bigr2id_news, max_beams=10).replace(' <end>', '...').replace('<start1> <start2> ', ''))

по предварительным данным причиной пожара стало короткое замыкание в электропроводке загорелась торговая палатка огонь затем перекинулся на лесной опушке диаметром около километра восточнее сержень-юрта вчечне третьи сутки идет уничтожение этого бандформирования заявили военные...
как сообщает риа новости...
основной темой предстоящих бесед станет ход ближневосточного мирного процесса целиком ложится на нынешнее руководство mi 5 стелла римингтон закончила писать воспоминания о шерлоке холмсе запрещен для показа в кинотеатре ударник...


по предварительным данным причиной пожара стало короткое замыкание в электропроводке загорелась торговая палатка огонь затем перекинулся на лесной опушке диаметром около километра восточнее сержень-юрта вчечне третьи сутки идет уничтожение этого бандформирования заявили военные...

как сообщает риа новости...

основной темой предстоящих бесед станет ход ближневосточного мирного процесса целиком ложится на нынешнее руководство mi 5 стелла римингтон закончила писать воспоминания о шерлоке холмсе запрещен для показа в кинотеатре ударник...


In [ ]:
for _ in range(3):
    print(generate_with_beam_search(matrix_news, id2word_news, bigr2id_news, max_beams=10, start='международный форум').replace(' <end>', '...').replace('<start1> <start2> ', ''))

международный форум начинает работу третья министерская конференция всемирной торговой организации wto org выпуск и распространение вредоносных программ tribe flood network tfn и stacheldraht известны с прошлого лета на 14 центов до 28,92 долл за баррель...
международный форум на высшем уровне между седьмым и восьмым кандидатами которые официально начали собирать нефть с поставкой сразу зафиксированы на алтуфьевском шоссе бибиревской улице волоколамское шоссе улице свободы улице генерала кузнецова в доверенности не было...
международный форум где планируется проводить мониторинг всего траффика проходящего через данного провайдера...


международный форум начинает работу третья министерская конференция всемирной торговой организации wto org выпуск и распространение вредоносных программ tribe flood network tfn и stacheldraht известны с прошлого лета на 14 центов до 28,92 долл за баррель...

международный форум на высшем уровне между седьмым и восьмым кандидатами которые официально начали собирать нефть с поставкой сразу зафиксированы на алтуфьевском шоссе бибиревской улице волоколамское шоссе улице свободы улице генерала кузнецова в доверенности не было...

международный форум где планируется проводить мониторинг всего траффика проходящего через данного провайдера...


In [ ]:
for _ in range(3):
    print(generate_with_beam_search(matrix_news, id2word_news, bigr2id_news, max_beams=10, start='быстрый рост').replace(' <end>', '...').replace('<start1> <start2> ', ''))

быстрый рост курса иены обусловили снижение цен фьючерсов на сырую нефть достигла рекордной для второго квартала уже будет составлен свой маркетинговый профайл после матчей сша словакия чехия латвия швейцария канада швеция финляндия...
быстрый рост и размножение раковых клеток прекращается и они сейчас унитожаются огнем артиллерии уничтожены такжедва автомобиля уаз убиты восемь сербов включая четырехлетнего ребенка и ранив двух милиционеров пустыми бутылками и банками из-под пива...
быстрый рост региональных банков...


быстрый рост курса иены обусловили снижение цен фьючерсов на сырую нефть достигла рекордной для второго квартала уже будет составлен свой маркетинговый профайл после матчей сша словакия чехия латвия швейцария канада швеция финляндия...

быстрый рост и размножение раковых клеток прекращается и они сейчас унитожаются огнем артиллерии уничтожены такжедва автомобиля уаз убиты восемь сербов включая четырехлетнего ребенка и ранив двух милиционеров пустыми бутылками и банками из-под пива...

быстрый рост региональных банков...


ВЫВОД

generate_with_beam_search работает лучше.

Оказалось, его можно даже немного настроить, чтобы предложения были не слишком короткие (это была основная проблема)